# imgtk

> Tools to manipulate images

In [ ]:
# | default_exp imgtk

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export
from itertools import product
from pathlib import Path
from typing import *

import matplotlib.pyplot as plt
import pandas as pd
import pyautogui
import pyperclip
from fastcore.test import *
import fire
from matplotlib.image import imread
from PIL import Image
from pillow_heif import register_heif_opener

In [ ]:
# | export
register_heif_opener()

In [ ]:
# | export


def fmter(
    src: Union[str, Path],  # source image path # fmt: skip
    fmt: str = "png",  # target image format # fmt: skip
) -> None:
    """Convert image format"""
    src = Path(src)
    trg = src.parent / f"{src.stem}.{fmt}"
    Image.open(src).save(trg)

Test file is created by `fmter`

In [ ]:
dir_img = Path("images").absolute()
if (dir_img / "mole.png").exists():
    (dir_img / "mole.png").unlink()
fmter(dir_img / "mole.jpg", "png")
test_eq((dir_img / "mole.png").exists(), True)

Test saved image has the same size as the original one

In [ ]:
test_eq(imread(dir_img / "mole.jpg").shape, imread(dir_img / "mole.png").shape)

In [ ]:
# | export


def recolor(
    src: str,  # source image path
    trg: str,  # target image path
    color: tuple[int, int, int],  # color to recolor to
) -> None:
    """Recolor image except pixels with alpha=0"""
    src, trg = Path(src), Path(trg)
    img = Image.open(src)
    width, height = img.size
    for x in range(width):
        for y in range(height):
            rgba = img.getpixel((x, y))
            if rgba[-1] != 0:
                img.putpixel((x, y), color)
    img.save(trg)

Test file is created by `recolor`

In [ ]:
rgb = (0, 0, 255)
blue_png = dir_img / "mole-blue.png"
if blue_png.exists():
    blue_png.unlink()
recolor(dir_img / "mole-no-background.png", blue_png, rgb)
test_eq(blue_png.exists(), True)

Test `rgb` is equal to the most common pixel color in the target image.

In [ ]:
img = Image.open(blue_png)
pixels = [img.getpixel((i, j)) for i, j in product(*map(range, img.size))]
pixel = pd.Series(pixels).value_counts().index[0][:3]
test_eq(pixel, (0, 0, 255))

In [ ]:
# | export
# | hide


def pixel_rgb() -> Tuple[int, int, int]:
    """Return the rgb of current mouse cursor position."""
    return tuple(pyautogui.pixel(*pyautogui.position()))


def rgb2hex(r: int, g: int, b: int) -> str:
    return f"#{r:02X}{g:02X}{b:02X}"


def pixel_hex():
    """Return the hex of current mouse cursor position."""
    return rgb2hex(*pixel_rgb())

In [ ]:
# | export
def pixel_color(
    fmt: str = "rgb",  # format of retured color (rgb or hex) # fmt: skip
) -> None:
    """Add the color of current mouse cursor position to the clipboard."""
    try:
        color = {"rgb": pixel_rgb, "hex": pixel_hex}[fmt]()
    except KeyError:
        raise ValueError(f"invalid format: {fmt}")
    pyperclip.copy(str(color))

In [ ]:
test_eq(rgb2hex(*(24, 24, 24)), "#181818")
# test_fail(lambda: pixel_color('wav'))

In [ ]:
# | export
def main():
    fire.Fire({"fmter": fmter, "pixel_color": pixel_color, "recolor": recolor})


if __name__ == "__main__":
    main()

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()